<a href="https://colab.research.google.com/github/ankesh86/PySparkNotebooks/blob/main/MLlib_Packages_chi_sq_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark==3.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317122 sha256=8361c4733902d3fa1378577e64744a96d3784e2a01abb77db99862fdbcf9c370
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basic_stats').getOrCreate()


In [3]:
ls sample_data

anscombe.json*               california_housing_train.csv  mnist_test.csv         README.md*
california_housing_test.csv  chi_sq.csv                    mnist_train_small.csv  Salary_Data.csv


In [4]:
df = spark.read.csv('sample_data/Salary_Data.csv',header = True, inferSchema = True)
df.count()

30

In [5]:
df.show()

+---------------+-------+
|YearsExperience| Salary|
+---------------+-------+
|            1.1|39343.0|
|            1.3|46205.0|
|            1.5|37731.0|
|            2.0|43525.0|
|            2.2|39891.0|
|            2.9|56642.0|
|            3.0|60150.0|
|            3.2|54445.0|
|            3.2|64445.0|
|            3.7|57189.0|
|            3.9|63218.0|
|            4.0|55794.0|
|            4.0|56957.0|
|            4.1|57081.0|
|            4.5|61111.0|
|            4.9|67938.0|
|            5.1|66029.0|
|            5.3|83088.0|
|            5.9|81363.0|
|            6.0|93940.0|
+---------------+-------+
only showing top 20 rows



In [6]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=df.columns, outputCol="features")
df_new = assembler.transform(df)
df_new.show()

+---------------+-------+-------------+
|YearsExperience| Salary|     features|
+---------------+-------+-------------+
|            1.1|39343.0|[1.1,39343.0]|
|            1.3|46205.0|[1.3,46205.0]|
|            1.5|37731.0|[1.5,37731.0]|
|            2.0|43525.0|[2.0,43525.0]|
|            2.2|39891.0|[2.2,39891.0]|
|            2.9|56642.0|[2.9,56642.0]|
|            3.0|60150.0|[3.0,60150.0]|
|            3.2|54445.0|[3.2,54445.0]|
|            3.2|64445.0|[3.2,64445.0]|
|            3.7|57189.0|[3.7,57189.0]|
|            3.9|63218.0|[3.9,63218.0]|
|            4.0|55794.0|[4.0,55794.0]|
|            4.0|56957.0|[4.0,56957.0]|
|            4.1|57081.0|[4.1,57081.0]|
|            4.5|61111.0|[4.5,61111.0]|
|            4.9|67938.0|[4.9,67938.0]|
|            5.1|66029.0|[5.1,66029.0]|
|            5.3|83088.0|[5.3,83088.0]|
|            5.9|81363.0|[5.9,81363.0]|
|            6.0|93940.0|[6.0,93940.0]|
+---------------+-------+-------------+
only showing top 20 rows



### Pearson Cofficient of Correlation

In [7]:
from pyspark.ml.stat import Correlation
pearson_corr = Correlation.corr(df_new,'features')
pearson_corr.show(2,False)

+----------------------------------------------------------------------------------+
|pearson(features)                                                                 |
+----------------------------------------------------------------------------------+
|1.0                 0.9782416184887599  \n0.9782416184887599  1.0                 |
+----------------------------------------------------------------------------------+



### Spearman Cofficient of Correlation

In [8]:
spearman_corr = Correlation.corr(df_new,'features',"spearman")
spearman_corr.show(2,False)

+----------------------------------------------------------------------------------+
|spearman(features)                                                                |
+----------------------------------------------------------------------------------+
|1.0                 0.9568313543516996  \n0.9568313543516996  1.0                 |
+----------------------------------------------------------------------------------+



## Chi-Squate Test

In [14]:
df = spark.read.csv('sample_data/chi_sq.csv',inferSchema=True,header=True)
df.count()

45211

In [15]:
df.show()

+--------+-------+-----+
| marital|housing|label|
+--------+-------+-----+
| married|    yes|    0|
|  single|    yes|    0|
| married|    yes|    0|
| married|    yes|    0|
|  single|     no|    0|
| married|    yes|    0|
|  single|    yes|    0|
|divorced|    yes|    0|
| married|    yes|    0|
|  single|    yes|    0|
|divorced|    yes|    0|
|  single|    yes|    0|
| married|    yes|    0|
| married|    yes|    0|
| married|    yes|    0|
| married|    yes|    0|
|  single|    yes|    0|
| married|    yes|    0|
| married|    yes|    0|
| married|    yes|    0|
+--------+-------+-----+
only showing top 20 rows



In [16]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline

# Assuming you already have a SparkSession instantiated and a DataFrame df loaded

# Configure an indexer for the marital status
marital_indexer = StringIndexer(inputCol="marital", outputCol="marital_num")
# Configure the OneHotEncoder to work on the indexed column
marital_encoder = OneHotEncoder(inputCols=["marital_num"], outputCols=["marital_vector"])

# Configure an indexer for the housing status
housing_indexer = StringIndexer(inputCol="housing", outputCol="housing_num")
# Configure the OneHotEncoder to work on the indexed column
housing_encoder = OneHotEncoder(inputCols=["housing_num"], outputCols=["housing_vector"])

# Build the pipeline
pipeline = Pipeline(stages=[marital_indexer, marital_encoder, housing_indexer, housing_encoder])

# Fit the pipeline to the data
model = pipeline.fit(df)

# Transform the data using the pipeline
df = model.transform(df)

# Show the transformed DataFrame
df.show()

+--------+-------+-----+-----------+--------------+-----------+--------------+
| marital|housing|label|marital_num|marital_vector|housing_num|housing_vector|
+--------+-------+-----+-----------+--------------+-----------+--------------+
| married|    yes|    0|        0.0| (2,[0],[1.0])|        0.0| (1,[0],[1.0])|
|  single|    yes|    0|        1.0| (2,[1],[1.0])|        0.0| (1,[0],[1.0])|
| married|    yes|    0|        0.0| (2,[0],[1.0])|        0.0| (1,[0],[1.0])|
| married|    yes|    0|        0.0| (2,[0],[1.0])|        0.0| (1,[0],[1.0])|
|  single|     no|    0|        1.0| (2,[1],[1.0])|        1.0|     (1,[],[])|
| married|    yes|    0|        0.0| (2,[0],[1.0])|        0.0| (1,[0],[1.0])|
|  single|    yes|    0|        1.0| (2,[1],[1.0])|        0.0| (1,[0],[1.0])|
|divorced|    yes|    0|        2.0|     (2,[],[])|        0.0| (1,[0],[1.0])|
| married|    yes|    0|        0.0| (2,[0],[1.0])|        0.0| (1,[0],[1.0])|
|  single|    yes|    0|        1.0| (2,[1],[1.0])| 

In [17]:
df_assembler = VectorAssembler(inputCols=['marital_vector','housing_vector'],outputCol='features')
df = df_assembler.transform(df)

df.show()

+--------+-------+-----+-----------+--------------+-----------+--------------+-------------+
| marital|housing|label|marital_num|marital_vector|housing_num|housing_vector|     features|
+--------+-------+-----+-----------+--------------+-----------+--------------+-------------+
| married|    yes|    0|        0.0| (2,[0],[1.0])|        0.0| (1,[0],[1.0])|[1.0,0.0,1.0]|
|  single|    yes|    0|        1.0| (2,[1],[1.0])|        0.0| (1,[0],[1.0])|[0.0,1.0,1.0]|
| married|    yes|    0|        0.0| (2,[0],[1.0])|        0.0| (1,[0],[1.0])|[1.0,0.0,1.0]|
| married|    yes|    0|        0.0| (2,[0],[1.0])|        0.0| (1,[0],[1.0])|[1.0,0.0,1.0]|
|  single|     no|    0|        1.0| (2,[1],[1.0])|        1.0|     (1,[],[])|[0.0,1.0,0.0]|
| married|    yes|    0|        0.0| (2,[0],[1.0])|        0.0| (1,[0],[1.0])|[1.0,0.0,1.0]|
|  single|    yes|    0|        1.0| (2,[1],[1.0])|        0.0| (1,[0],[1.0])|[0.0,1.0,1.0]|
|divorced|    yes|    0|        2.0|     (2,[],[])|        0.0| (1,[0]

In [18]:
from pyspark.ml.stat import ChiSquareTest
chi_sq = ChiSquareTest.test(df,"features","label").head()
print("pValues: " + str(chi_sq.pValues))

pValues: [1.0,1.0,1.0]


In [21]:
# Print the degrees of freedom
print("degreesOfFreedom: " + str(chi_sq.degreesOfFreedom))
print("statistics: " + str(chi_sq.statistics))
print("pValues: " + str(chi_sq.pValues))

degreesOfFreedom: [0, 0, 0]
statistics: [0.0,0.0,0.0]
pValues: [1.0,1.0,1.0]


#### **The DataFrame you've shown appears to be related to some preprocessing steps commonly used in machine learning to prepare categorical data for modeling. Let me break down the information you have provided:**

marital, housing: These columns represent categorical variables in their original string form.

label: This is typically the target variable you want to predict, with '0' possibly indicating a negative class and '1' indicating a positive class.

marital_num, housing_num: These columns represent the numerical encoding of the categorical variables. This is a necessary step because most machine learning algorithms work with numerical data, not strings. For instance, 'married' might be coded as 0.0, 'single' as 1.0, and 'divorced' as 2.0.

marital_vector, housing_vector: These columns represent the one-hot encoded vectors corresponding to the numerical encoding of the categorical variables. One-hot encoding is a process that converts categorical variables into a form that could be provided to ML algorithms to do a better job in prediction. It creates a binary column for each category and returns a sparse matrix or dense array (depending on the input data).

In marital_vector and housing_vector, the structure (2,[0],[1.0]) might be explained as follows:

2 indicates the size of the vector. It means there are two possible values.

[0] indicates the index of the array where the 1.0 is placed, which represents the presence of a particular category.

[1.0] indicates the value at that index position. A '1' in one of these positions indicates the presence of that particular category, while all other positions will be '0'.
This is the "one-hot" part: only one position is "hot" (1) for each observation.

features: This is typically a combined feature vector that would be used as input for machine learning models. It combines all the individual vectors into a single feature vector.

## **Why Use OneHotEncoder?**

OneHotEncoder is used because many machine learning algorithms cannot work directly with categorical data. If you encode the categories as integers, the model may misinterpret the data to be in some kind of order, 0 < 1 < 2, which is not the case.

The one-hot encoding creates a binary column for each category and returns a matrix with 1s and 0s. It's a way of telling the model "this data is categorical, not numerical".

Using OneHotEncoding prevents the model from assuming a natural ordering between categories which can result in poor performance or unexpected results (for example, it would be incorrect to think that 'married' < 'single' < 'divorced' just because 0 < 1 < 2).

### **In summary, the numerical transformation (marital_num, housing_num) is an intermediate step that is often followed by one-hot encoding (marital_vector, housing_vector) to create binary variables that represent the presence of each possible category. The final features column is a vector that includes all these binary variables as a single input feature for modeling purposes.**

# **Binarizer**

In [22]:
from pyspark.ml.feature import Binarizer
binarizer = Binarizer(threshold=0.99, inputCol="label", outputCol="binarized_label")
new_df = binarizer.transform(df)

IllegalArgumentException: Data type IntegerType is not supported.

# **Different pre-processing operations in ML**
* **Binarizer** - making values 0 or 1
* **Principal Component Analysis** - reduces the dimension
* **Normalizer** - mean 0 and SD 1
* **Standard Scaling** - normalise data between [0,1]
* **min-Max Scaling** - we can specify limits based on max and min in DF
* **MaxAbsScaler** - rescales feature values between -1 and 1
* **Binning** - bucketing, (Bucketizer) make data in category